<a href="https://colab.research.google.com/github/schwallergroup/ai4chem_course/blob/scikit_learn/notebooks/02%20-%20Supervised%20Learning/training_and_evaluating_ml_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 2 tutorial - AI 4 Chemistry

## Table of content

- Software
- Supervised learning
  - Common algorithms
  - Model evaluation and data splitting
  - Basic steps
- Hands-on exercises
  - Regression example


### TODO

# 0. Software
### Scikit-learn
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection, model evaluation, and many other utilities. We will learn to use scikit-learn to do machine learning work. You can also browse the scikit-learn [user guide](https://scikit-learn.org/stable/user_guide.html) and [tutorials](https://scikit-learn.org/stable/tutorial/index.html) for additional details.
### Essential Libraries and Tools 
Scikit-learn depends on two other Python packages, NumPy and SciPy. For plotting and interactive development, you should also install matplotlib, IPython, and the Jupyter Notebook.
- **NumPy** is one of the fundamental packages for scientific computing in Python. In scikit-learn, the NumPy array is the fundamental data structure. Any data you’re using will have to be converted to a NumPy array.
- **SciPy** is a collection of functions for scientific computing in Python. Scikit-learn draws from SciPy’s collection of functions for implementing its algorithms.
- **Matplotlib** is the primary scientific plotting library in Python. It provides functions for making publication-quality visualizations such as line charts, histograms, scatter plots, and so on.
- **Pandas** Python library for data wrangling and analysis. It can ingest from a great variety of file formats and databases, like SQL, Excel files, and comma-separated values (CSV) files.

### XGBoost
XGBoost (eXtreme Gradient Boosting) is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way. You can also browse the [XGBoost Documentation](https://xgboost.readthedocs.io/en/stable/) for additional details.

### DeepChem
DeepChem is a high quality open-source toolchain that democratizes the use of deep-learning in chemistry, biology and materials science. It also provides various tools for dataset loader, splitters, molecular featurization, model construction and hyperparameter tuning. You can also browse the [DeepChem Ducumentation](https://deepchem.readthedocs.io/en/latest/) for additional details.

We will first install the required libraries. We also need `RDKit` library to process and analyze molecules, like calculating molecular descriptors.

In [ ]:
!pip install numpy scipy matplotlib scikit-learn pandas rdkit xgboost deepchem

# 1. Supervised learning
Two major types of supervised machine learning problems:
- **Classification** task is to predict a class label, which is a choice from a predefined list of possibilities.
- **Regression** task is to predict a continuous number, or a floating-point number.

## 1.1 Common algorithms
- k-Nearest Neighbors (k-NN)
- Linear Models
- Support Vector Machines (SVM)
- Decision Trees
- Ensembles of Decision Trees
  - Random forests
  - Gradient boosting machines

We can use `scikit-learn` to create ML models of different algorithms.

In [ ]:
# linear regressor
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()

**Exercise**: Create a k-NN classifier using scikit-learn. [Tip](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [ ]:
### YOUR CODE

# import
' your code '

# create knn model
knn_clf = ' your code '

### END

Next time, you can browse the scikit-learn [user guide](https://scikit-learn.org/stable/user_guide.html) to learn about supported algorithms and how to create the model you want.

## 1.2 Model evaluation and data splitting
### Why do we need to split dataset?
We want models learn from data to predict on new data, but we cannot use the data we used to build the model to evaluate it. Hence, you need to separate your input data into `training`, `validation`, and `test` subsets to prevent your model from overfitting and to evaluate your model effectively. If you have more time, you can read this [article](https://towardsdatascience.com/how-to-split-data-into-three-sets-train-validation-and-test-and-why-e50d22d3e54c) for more details.
### Evaluation metrics
The metrics used to evaluate the ML models are very important. The choice of metrics to use influences how model performance is measured and compared. The main evaluation metrics for regression and classification tasks are illustrated below. If you have more time, you can read this [article](https://blog.knoldus.com/model-evaluation-metrics-for-machine-learning-algorithms/) for more details.

<div align="center">
<img src="https://www.oreilly.com/api/v2/epubs/9781492073048/files/assets/mlbf_0407.png" width="500"/>
</div>


## 1.3 Basic steps
0. Define the task
1. Prepare data & split data
2. Choose the model
3. Train the model
4. Evaluate the model
5. Use the model

# 2. Hands-on exercises
## 2.1 Regression example
Below is a simple example to show basic steps of regression tasks. **Our goal** is to build a ML model that can learn from chemical structures (as encoded in SMILES strings) to predict **water solubility**. We will use ESOL dataset from [MoleculeNet](https://doi.org/10.1039/C7SC02664A) to train the models. This dataset contains structures and water solubility data for 1128 compounds.

Load dataset & show data

In [ ]:
import pandas as pd

# load dataset from a CSV file
esol_df = pd.read_csv('data/esol.csv')
esol_df


The original dataset contains 2 columns, where the `smiles` column represents the SMILES strings of the solute molecules. The column `log solubility (mol/L)` represents the solubility of molecules in water, which is the predicted target of our task.

In [ ]:
# Get NumPy arrays from DataFrame for the input and target
smiles = esol_df['smiles'].values
y = esol_df['log solubility (mol/L)'].values

We need to convert the SMILES strings of molecules into numerical values that can be used as input to the ML models. We can calculate molecular descirptors from SMILES strings by some software like `RDKit`, `DeepChem` and [Mordred](https://github.com/mordred-descriptor/mordred). Here we use DeepChem [Featurizers](https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html) to compute molecular descriptors.

In [ ]:
# Here, we use molecular descriptors from RDKit, like molecular weight, number of valence electrons, maximum and minimum partial charge, etc.
from deepchem.feat import RDKitDescriptors
featurizer = RDKitDescriptors()
features = featurizer.featurize(smiles)
print("Number of molecular descriptors:", features.shape[1])

**Exercise**: You can try to use [MACCSKeysFingerprint](https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html#maccskeysfingerprint) in DeepChem [Featurizers](https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html) to compute molecular descriptors.

In [ ]:
### YOUR CODE

# import
' your code '

# create featurizer
mf_featurizer = ' your code '

# compute molecular descriptors
mf_features = ' your code '

# show results
print("Number of molecular descriptors (MACCS Fingerprint):", mf_features.shape[1])

### END

Data preprocessing

In [ ]:
import numpy as np

# Min-Max Normalization of features
fea_max = features.max(axis=0)
fea_min = features.min(axis=0)
fea_norm = (features - fea_min) / (fea_max - fea_min)

# Check if normalized features contain invalid values
contain_nan = (True in np.isnan(fea_norm))
if contain_nan:
    print('Our normalized features contain invalid values, please delete them before model training!')
    fea_norm = fea_norm[:, ~np.isnan(fea_norm).any(axis=0)]
    print('Dropping of columns containing invalid values has been completed.')
else:
    print('Our normalized features do not contain invalid values.')
print("Shape of molecular descriptors after data preprocessing:", fea_norm.shape[1])

Dataset split

In [ ]:
from sklearn.model_selection import train_test_split
X = fea_norm
# training data size : test data size = 0.8 : 0.2
# fixed seed using the random_state parameter, so it always has the same split.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=0)

Create models

In [ ]:
# random forest regressor, and the default criterion is mean squared error (MSE)
from sklearn.ensemble import RandomForestRegressor
ranf_reg = RandomForestRegressor(n_estimators=10, random_state=0)  # using 10 trees and seed=0

# XGBoost regressor
from xgboost import XGBRegressor
bst_reg = XGBRegressor(n_estimators=10, random_state=0)  # using 10 trees and seed=0

Train and evaluate the models
- Mean Squared Error: $MSE$ = $\frac{1}{n} \Sigma_{i=1}^n({y}-\hat{y})^2$
- Root Mean Squared Error: $RMSE$ = $\sqrt{MSE}$ = $\sqrt{\frac{1}{n} \Sigma_{i=1}^n({y}-\hat{y})^2}$

We choose `RMSE` (lower is better) as the evaluation metric for this task.

In [ ]:
from sklearn.metrics import mean_squared_error

# for random forests
ranf_reg.fit(X_train, y_train)  # train the model
y_pred_train = ranf_reg.predict(X_train)
y_pred_test = ranf_reg.predict(X_test)
ranf_train_mse = mean_squared_error(y_train, y_pred_train)
ranf_test_mse = mean_squared_error(y_test, y_pred_test)
ranf_train_rmse = ranf_train_mse ** 0.5
ranf_test_rmse = ranf_test_mse ** 0.5
print('Random forests performance:')
print('RMSE on train set: {:.3f}, and test set: {:.3f}.\n'.format(ranf_train_rmse, ranf_test_rmse))

# for XGBoost
bst_reg.fit(X_train, y_train)  # train the model
y_pred_train = bst_reg.predict(X_train)
y_pred_test = bst_reg.predict(X_test)
bst_train_mse = mean_squared_error(y_train, y_pred_train)
bst_test_mse = mean_squared_error(y_test, y_pred_test)
bst_train_rmse = bst_train_mse ** 0.5
bst_test_rmse = bst_test_mse ** 0.5
print('XGBoost performance:')
print('RMSE on train set: {:.3f}, and test set: {:.3f}.'.format(bst_train_rmse, bst_test_rmse))

The results show that the RMSE value of random forests on the test set is smaller.

**Exercise**: Try to train a SVM model and evaluate it. [Tip](https://scikit-learn.org/stable/modules/svm.html#regression)

In [ ]:
### YOUR CODE

# import
' your code '

# create a model
svm_reg = ' your code '

# train the model
' your code '

# evaluate the model
' your code '
svm_train_rmse = ' your code '
svm_test_rmse = ' your code '

# print results
print('SVM performance:')
print('RMSE on train set: {:.3f}, and test set: {:.3f}.'.format(svm_train_rmse, svm_test_rmse))

### END

Perfect! Now you have mastered training and evaluating the model you want.

Cross-validation and hyperparamter optimization

In [ ]:
from sklearn.model_selection import cross_val_score

best_score = 0

for n_estimators in [5, 10, 15, 20]:
    for max_features in ['auto', 'sqrt', 'log2']:
        # for each combination of parameters, train a random forest model
        rf_gs = RandomForestRegressor(n_estimators=n_estimators, max_features=max_features, random_state=0)
        # perform cross-validation
        scores = cross_val_score(rf_gs, X_train, y_train, cv=3)
        # compute mean cross-validation accuracy
        score = np.mean(scores)
        # if we got a lower MSE (better performance), store the score and parameters
        if score > best_score or best_score == 0:
            best_score = score
            best_parameters = {'max_features': max_features, 'n_estimators': n_estimators}
# re-train a model using best hyperparameters
rf_gs = RandomForestRegressor(**best_parameters, random_state=0)
rf_gs.fit(X_train, y_train)
y_pred_test = rf_gs.predict(X_test)
rf_gs_mse = mean_squared_error(y_test, y_pred_test)
rf_gs_rmse = rf_gs_mse ** 0.5
print('Best paramter: ', best_parameters)
print('Random forests performance after hyperparamter optimization:')
print('RMSE on test set: {:.3f}.\n'.format(rf_gs_rmse))

In [ ]:
# another method
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [5, 10, 15, 20],
              'max_features': ['auto', 'sqrt', 'log2']}

grid_search = GridSearchCV(RandomForestRegressor(random_state=0), param_grid, cv=3)
grid_search.fit(X_train, y_train)

# re-train a model using best hyperparameters
rf_gs = RandomForestRegressor(**grid_search.best_params_, random_state=0)
rf_gs.fit(X_train, y_train)
y_pred_test = rf_gs.predict(X_test)
rf_gs_mse = mean_squared_error(y_test, y_pred_test)
rf_gs_rmse = rf_gs_mse ** 0.5
print('Best paramter: ', grid_search.best_params_)
print('Random forests performance after hyperparamter optimization:')
print('RMSE on test set: {:.3f}.\n'.format(rf_gs_rmse))

Now, we get a better random forest model (0.699 < 0.705).

# Introduction to traditional ML.

## Supervised learning

Training a model to take inputs X and return output y.

As you have seen in class, for this type of learning, we have two variants:

- Classification
- Regression

Linear regression is one example of suppervised learning for regression.

# Regression

### TODO: Improve this introduction based on the ESOL paper, why is solub. prediction important?

One problem in both academic and industrial chemistry is predicting solubility. For instance we might know that some molecule has good potential as a ligand for some relevant reaction, however when you synthesize it, you realize it's not soluble under your already optimized reaction conditions! 😥

It would be extremely useful to know the solubility of my molecule, **before I even try to synthesize it**!

---

In this task we will try to solve this using supervised learning. In particular, we will train a regression model using the very convenient [scikit-learn](https://www.kaggle.com/competitions/MerckActivity/data) Python library, to predict solubility based on some molecular descriptors.

In [ ]:
# TODO: Let's start with loading the data, visualizing some molecules and their solubility
# Let's also see some stats. e.g. size of dataset, distribution of solubility, etc.

# TODO: Generate features
# TODO do a train/test split

In [ ]:
# TODO: Let's give an introduction to scikit learn by doing a simple linear regression and see the results.
# TODO Introduce sckit-learn, and use a RF model for this.
import tempfile
from sklearn.manifold import Isomap
from sklearn.neighbors import KNeighborsTransformer
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_regression
cache_path = tempfile.gettempdir()  # we use a temporary folder here
X, _ = make_regression(n_samples=50, n_features=25, random_state=0)
estimator = make_pipeline(
    KNeighborsTransformer(mode='distance'),
    Isomap(n_components=3, metric='precomputed'),
    memory=cache_path)
X_embedded = estimator.fit_transform(X)
X_embedded.shape

In [ ]:
# TODO: Let's train another model (this can be an excercise)

# EXERCISE: Implement random forest and and XGBoost models using scikit learn

# As we see, there are many possible models we can use for this task. But which one is better?

In addition, each model has a set of hyperparameters that we need to tune ourselves. How do we select them?

This is an important part of machine learning! What we want to know is: What is the best combination of model + model hyperparameters for our task? 
As you've seen in the course, common strategies to evaluate and compare model's performance include:

- Splitting dataset in train/validation/test.
- Doing cross-validation for hyperparameter tuning.

In [ ]:
# TODO: Split data in train/valid/test

# Retrain the models on the train set, and compare them using the validation set.

# What model is the best?

In [ ]:
# TODO: Let's do cross-validation

# Optimize the hyperparameters for XGBoost, and again compare performance

In [ ]:
# TODO: Finally, compare all models on the test set.
# Explain that test set should never be seen by models.
# This is all completely new data so we know how it would work in real life.

---

# Classification

We now turn our attention towards the other type of supervised learning: classification.

Many questions in chemistry can be framed as a classification task: 

- Will this molecule act as a nucleophile or electrophile in my reaction?
- What is the smell of this substance? (fruity, citrus, sweet, ...)

But in this tutorial we will try to respond:

<div>
<img src="img/is_this_toxic.png" width="500"/>
</div>


---

## For this, we need data. [MoleculeNet](https://moleculenet.org/datasets-1) provides several datasets, and we'll work with `ClinTox` for prediction of toxicity.

ClinTox is a dataset containing `qualitative data of drugs approved by the FDA and those that have failed clinical trials for toxicity reasons`.

Let's see if one of our models can tell what molecules are toxic!

This is super useful for instance in drug discovery, where we want to know if a molecule has potential as a drug, **even before we synthesize it**.

The steps we follow are similar to those we saw for regression:

1. Prepare & split data
2. Choose a model
3. Train the model
4. Evaluate the model
5. Use the model

In [ ]:
import pandas as pd

# Load clintox data from the data directory and see what it contains
df_toxicity = pd.read_csv("data/toxcast_data.csv")
df_toxicity.head(3)

This dataset contains the molecule SMILES, plus a lot of data coming from different [toxicity measurements](https://www.epa.gov/chemical-research/exploring-toxcast-data).

We will stick to `TOX21_TR_LUC_GH3_Antagonist`, as is the one for which we have more data.

In [ ]:
df_tox = df_toxicity.loc[:,["smiles","TOX21_TR_LUC_GH3_Antagonist"]].dropna()
df_tox.columns = ["smiles", "toxic"]
df_tox.head()

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

# Visualize some of the molecules of this dataset
n=6
smiles = df_tox.smiles.sample(n).values
legend = df_tox.toxic.sample(n).values
molecs = [Chem.MolFromSmiles(s) for s in smiles]

Draw.MolsToGridImage(
    molecs,
    subImgSize=(600,300),
    legends=["Toxic" if i==1 else "Non toxic" for i in legend]
)

In [ ]:
# How many toxic molecules are in the dataset?
counts = df_tox["toxic"].value_counts()

print(f"The dataset contains {counts.sum()} molecules; {counts.iloc[1]} of them are toxic.")

## Now, we will calculate some molecular descriptors using the [mordred package](http://mordred-descriptor.github.io/documentation/master/descriptors.html).

This can take a while (how much??), you can use this time to explore a bit the more than 1600 descriptors from mordred!

In [ ]:
from rdkit import Chem
from deepchem.feat import MordredDescriptors

featurizer = MordredDescriptors(ignore_3D=True)
features = featurizer.featurize("CCC")
print("Number of molecular descriptors:", features.shape[1])

X_raw = df_tox.smiles.apply(lambda x: featurizer.featurize(x))
y_raw = df_tox.toxic

In [ ]:
# As you see from the warnings, mordred couldn't calculate features for a few molecules (but don't worry!)
# Remove these molecules from the dataset
import numpy as np

# Featurizer should return array of size (1, number_of_features)
missing = X_raw.apply(lambda x: x.shape == (1, features.shape[1]))

print(f"Dropping {(~missing).sum()} molecules that couldn't be featurized.")
X = X_raw[missing].values
y = y_raw[missing].values

X = np.concatenate(X)

# Save
np.save("data/features_tox.npy", X)

In [ ]:
X = np.load("data/features_tox.npy")

In [ ]:
# Challenge: Which molecules couldn't be featurized? Why?
# Using code from above, visualize the faulty molecules.

##################
# Your code here #
##################

## Data splitting.

For this exercise, we will do a simple train/test split as we will not optimize hyperparameters. (Maybe bonus exercise here?)

In [ ]:
from sklearn.model_selection import train_test_split

# train data size : test data size = 0.8 : 0.2
# fixed seed using the random_state parameter, so it always has the same split.
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.8,
    random_state=0
)

print(f"Train set size is {X_train.shape[0]} rows, test set size is {X_test.shape[0]} rows.")

## Model

Let's train a Random Forest Classification from [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(max_depth=10, random_state=0)
rf_clf.fit(X_train, y_train);

### Exercise: You have already seen some cool classification algorithms in class.

In this exercise, your task is to implement your 2 favorite algorithms using sklearn. 

Recommendations: 

- You can choose from [Logistic Regression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), [Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html#classification), [Gradient Boosting](https://scikit-learn.org/stable/modules/ensemble.html#gradient-tree-boosting), or any other from the [sklearn documentation](https://scikit-learn.org/stable/supervised_learning.html).
- Give a different name to each model. For instance, our Random Forest model is `rf_clf`.

In [ ]:
##################
# Your code here #
##################

## After training these models, let's see which one worked best! 

For the evaluation of classification models, **we use different metrics** than evaluation of regression models. \
You can read more about each metric [here](https://scikit-learn.org/stable/modules/model_evaluation.html), but for this tutorial we will use [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score), [ROC-AUC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score), and [F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score).

In [ ]:
# Let's evaluate our models 
y_pred_rf = rf_clf.predict(X_test)

# Exercise: Use your models to predict the toxicity of the molecules on the test set.

##################
# Your code here #
##################

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    f1_score
)

# Let's calculate accuracy_score for all our models
acc_rf = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy of Random Forest Classifier is {acc_rf:.3f}")
auc_rf = roc_auc_score(y_test, y_pred_rf)
print(f"ROC-AUC of Random Forest Classifier is {auc_rf:.3f}")
f1s_rf = f1_score(y_test, y_pred_rf)
print(f"F1 Score of Random Forest Classifier is {f1s_rf:.3f}")

# Exercise: Calculate the 3 metrics for every model you trained, and compare results

##################
# Your code here #
##################

---
Find out what each metric is telling us. Should we trust such high accuracies? Why is accuracy so high compared to the other metrics?

YOUR ANSWER:

---

In [ ]:
# TODO let's also do confusion matrix
from pycm import ConfusionMatrix

cm = ConfusionMatrix(actual_vector=y_test, predict_vector=y_pred_rf)
cm.relabel(mapping={0:"Non Toxic", 1:"Toxic"})
cm.plot(number_label=True);

### Some models, like Random Forest, are inherently interpretable, so we can easily see what features are important:

---
What can we learn from these results? \
Go to [Mordred documentation](http://mordred-descriptor.github.io/documentation/master/descriptors.html) and find these features. What are they, and do they make any sense?

### Discuss

---

# So after all what, is my molecule toxic or what? Let's see what our model says!

In [ ]:
# Define molecule here
molecule = "O=C1N(C)C(C2=C(N=CN2C)N1C)=O"
mol = Chem.MolFromSmiles(molecule)

# Calculate features
X_my_mol = featurizer.featurize(molecule)

# Get model prediction
is_toxic = rf_clf.predict(X_my_mol)
is_toxic = "This molecule is toxic!" if is_toxic else "This is not toxic :)"

Draw.MolsToGridImage(
    [mol],
    subImgSize=(600,300),
    legends=[is_toxic],
    molsPerRow=1
)

---

## Should I trust this though?: Interpretability and explainability.

> Cool, our models know stuff, but we also want to know! \
> What do they look at when they predict toxicity? Is there a key feature?

\
Model explainability is a critical component of machine learning that seeks to provide insights into how a model arrives at its predictions or decisions. In other words, it aims to make the "black box" of machine learning models more transparent, so that we can understand the factors that are driving the model's output.

There are many different methods for achieving model explainability (more on this [here](https://www.kaggle.com/learn/machine-learning-explainability)). \
These techniques can help us identify which features or variables are most important in driving the model's output, and can provide insights into the model's decision-making process.

### Let's explore ways of measuring feature importance, which will tell us what our models are looking out when making predictions.

In [ ]:
importances = pd.Series(rf_clf.feature_importances_, name="importance")
std = pd.Series(np.std([tree.feature_importances_ for tree in rf_clf.estimators_], axis=0), 
                name="std")

importances = pd.concat([importances, std], axis=1)
importances = importances.sort_values(by="importance", ascending=False).iloc[:20]


import matplotlib.pyplot as plt

fig, ax = plt.subplots()
importances["importance"].plot.bar(yerr=importances["std"], ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
# TODO train another model (maybe XGBoost) and explain using SHAP
# Already too much for 2 hours, but can be left to read more

---
---

# Extra stuff, dev.

# Tasks for today: 

- scikit learn
- classification
- regression: [ESOL dataset](https://www.kaggle.com/competitions/MerckActivity/data)
- XGBoost: [Merck kaggle challenge](https://www.kaggle.com/competitions/MerckActivity/data)
- SHAP values + feature importance
- Evaluation of ML models
- Cross-validation: hyperparameter tuning
- Train/valid/test split

In [ ]:
d = pd.read_csv("data/toxcast_data.csv")
num_nans = (~d.isna()).sum(axis=0).sort_values(ascending=False)
d.iloc[:,1:].apply(lambda x: x.value_counts(dropna=False)).T.loc[num_nans.index[1:]].head(30)

# We'll play with balanced datasets. 

- TOX21_Aromatase_Inhibition 	6674 	1276
- TOX21_ERa_LUC_BG1_Agonist 	6691 	1259
- TOX21_TR_LUC_GH3_Antagonist 	6186 	1764